## Reading different types of Data, Partitions, Parellization
##### DATAFRAME API IS PREFERRED against RDD API, as it is much faster. Datasets API is not avaliable in python.

#### 1. Dataframes are immutable ; with every transformation new dataset is created

#### 2. Spark datasets are represented as a list of entries.
       This list is broken into partitions stored on a different machines. 
       Each partition holds a unique subset of the entries in the list. 
       Spark calls these datasets "Resilient Distributed Datasets" (RDDs).
#### 3. At low level, everything is implemented as RDDs

#### 4. DataFrames are ultimately represented as RDDs, with additional meta-data.

#### 5.When you create a DataFrame, this collection is going to be parallelized

#### 6.Spark DataFrames schemas are defined as a collection of typed columns. The entire schema is stored as a StructType and individual columns are stored as StructFields.

## Spark supports text files, SequenceFiles, and any other Hadoop InputFormat.
####  Hadoop format 
1. CSV Files
2. Text Files
3. JSON Records
4. Avro Files
5. Sequence Files
6. RC Files
7. ORC Files
8. Parquet Files

In [32]:
from pyspark.sql import SparkSession
import pandas as pd

In [17]:
spark = SparkSession.Builder().appName("fileformats").getOrCreate()

### 1. Text/CSV files

In [72]:
#Method 1
pandas_df = pd.read_csv("Data/Employee_Statistics.csv")

In [71]:
pandas_df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                      int64
dtype: object

In [64]:
from pyspark.sql.types import *

    DataType
            ArrayType
            MapType
            NullType
            StructField
            StructType
    AtomicType(DataType)
        BinaryType
        BooleanType
        DateType
        StringType
        TimestampType
    FractionalType(NumericType)
        DecimalType
        DoubleType
        FloatType
    IntegralType(NumericType)
        ByteType
        IntegerType
        LongType
        ShortType

In [69]:
# StructField objects are created with the name, dataType, and nullable properties
schema = StructType([StructField("Customer", StringType(), True)\
                   ,StructField("Month", DateType(), True)\
                   ,StructField("Amount", IntegerType(), True)])

In [108]:
schema = StructType([StructField("enrollee_id", IntegerType(), False)\
                    ,StructField("city", StringType(), True)\
                    ,StructField("city_development_index", FloatType(), True)\
                    ,StructField('gender', StringType(), True)\
                    ,StructField('relevent_experience', StringType(), True)\
                    ,StructField('enrolled_university', StringType(), True)\
                    ,StructField('education_level', StringType(), True)\
                    ,StructField('major_discipline', StringType(), True)\
                    ,StructField('experience', StringType(), True)\
                    ,StructField('company_size', StringType(), True)\
                    ,StructField('company_type', StringType(), True)\
                    ,StructField('last_new_job', StringType(), True)\
                    ,StructField('training_hours', IntegerType(), True),StructField('target', IntegerType(), True)])

In [109]:
df1= spark.createDataFrame(pandas_df, schema=schema)

In [110]:
#Method 2
df2= (spark.read.format("csv").options(header="true").load("Data/Employee_Statistics.csv"))

In [113]:
## Let us compare method 2 & method 1
## In Method 2: whole data is loaded as string
## Method 1 give us more control over describing data types for columns

In [114]:
df2.printSchema()

root
 |-- enrollee_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_development_index: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- relevent_experience: string (nullable = true)
 |-- enrolled_university: string (nullable = true)
 |-- education_level: string (nullable = true)
 |-- major_discipline: string (nullable = true)
 |-- experience: string (nullable = true)
 |-- company_size: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- last_new_job: string (nullable = true)
 |-- training_hours: string (nullable = true)
 |-- target: string (nullable = true)



In [115]:
df1.rdd.getNumPartitions()

8

In [116]:
df2.rdd.getNumPartitions()

1

In [ ]:
## Method 1 give me default data partitions done while dataframe creation, but partitions are not created in method 2